In [1]:
import os

from datetime import datetime, date
from uuid import uuid4
from dateutil.tz import tzlocal
from ast import literal_eval

import numpy as np
import pandas as pd
import warnings
import librosa
import pickle

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject
from pynwb.epoch import TimeIntervals
from pynwb.image import ImageSeries
from ndx_manoli_meta import AssayMetadata

from nwb_utils import get_date_from_block
from behavior_error_checks import *

In [2]:
# ----- Set up user parameters
directory_keyword = 'PPTs' # how the assay directories are labeled
metadata_file = 'metadata_ppts_v2.csv' # where the initial metadata is stored
boris_keyword = 'PPT'
score_path = os.path.join('Scn2a_X1Behavior_Nov22','Aggregated_Events')
assayregex = 'P|p'
# colors for plotting
fillcols = [[160, 146, 95],[245, 201, 39],[89, 91, 125],[63, 78, 245]]
linecols = [[96, 87, 57],[147, 120, 23],[53, 54, 75],[33, 41, 131]]

In [18]:
# ----- NWB set up -----

# 1. Set up user parameters
# 2. Loop over metadata file
# 3. Get and set up metadata
# 4. Set up behavior table
# 5. Write nwb file

# -- set parameters
lab = "Manoli @ UCSF"
exclude_flag = False
# path to storage for NWB files
nwbfile_path = os.path.join('M:\\','scn2a-paper-GWJSNH','NWB-files')
# whether to write NWB files to disk yet
write_NWB_to_disk = False

# load metadata
meta = pd.read_csv(metadata_file)
meta.FocalColor = meta.FocalColor.apply(literal_eval) # convert the colors to real arrays

# -- loop over metadata
for i, ptag in enumerate(meta.PairTag):
    assay_type = meta.AssayType[i]
    nwbfilename = f'{ptag}_{assay_type}.nwb'
    print(nwbfilename)
    
    # check if file already exists
    wfullpath = os.path.join(nwbfile_path,nwbfilename)    
    if not os.path.exists(wfullpath):    
        
        # get session specific metadata
        thisdate = str(meta.RecDate[i])
        pairdate = str(meta.PairDate[i])
        
        # calculate days post pairing
        rdate = date(int(thisdate[0:4]),int(thisdate[4:6]),int(thisdate[6:]))
        pdate = date(int(pairdate[0:4]),int(pairdate[4:6]),int(pairdate[6:]))
        dpp = rdate-pdate

        # set up recording time... it would be nice to get actual video data for the times
        datepieces = get_date_from_block(thisdate)
        rtime = meta.RecTime[i]
        timepieces = rtime.split(':')
        sess_start = datetime(datepieces[0],datepieces[1],datepieces[2],int(timepieces[0]),int(timepieces[1]),0,0,tzlocal())

        session_description = f'Behavioral annotations from pair {ptag} in a(n) {assay_type} assay.'

        # make NWB file
        nwbfile = NWBFile(
            session_description=session_description,
            identifier = str(uuid4()),
            session_start_time = sess_start,
            lab=lab,
            experimenter=meta.RanBy[i],
            session_id = nwbfilename[0:-4], # check this
        )

        # add subject info
        nwbfile.subject = Subject(
            subject_id = meta.FocalID[i],
            species = 'Microtus ochrogaster',
            sex = meta.FocalSex[i],
            genotype = meta.FocalGT[i]
        )

        # get assay duration
        duration = float(meta.AssayDuration[i])

        # figure out partner info
        if meta.FocalSex[i]=='F':
            pID = meta.MaleID[i]
            pGT = meta.MaleGT[i]
        elif meta.FocalSex[i]=='M':
            pID = meta.FemaleID[i]
            pGT = meta.MaleGT[i]
        else:
            print(f'Focal sex is neither F nor M; something is wrong with {ptag}.')

        # TODO convert these to date objects and make sure this works
        # figure out time since pairing
        days = meta.RecDate[i] - meta.PairDate[i]

        # add lab metadata
        metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=exclude_flag,
                        duration=duration,
                        room=str(meta.AssayRoom[i]),
                        timeline=str(meta.Timeline[i]),
                        ethogram=str(meta.Ethogram[i]),
                        experimenter=str(meta.ScoredBy[i]),
                        timeline_complete=meta.FullTimeline[i],
                        colors=meta.FocalColor[i][0],
                        assay_type__partner_GT='WT',
                        assay_type__description='Standard PPT (vocal series).',
                        assay_type__days_post_pairing=int(dpp.days),
                        assay_type__divided=False,
                        assay_type__stranger_GT='WT',
                        assay_type__PPT_lane=meta.PPTlane[i],
                        assay_type__partner_chamber=meta.PartnerChamber[i],
                        assay_type__left_GT=meta.LeftGT[i],
                        assay_type__left_sex=meta.LeftSex[i],
                        assay_type__right_GT=meta.RightGT[i],
                        assay_type__right_sex=meta.RightSex[i],
                        assay_type__left_type=meta.LeftType[i],
                        assay_type__right_type=meta.RightType[i],
                        assay_type__annotations=os.path.join(meta.ScorePath[i],meta.ScoreFile[i]),
                        )


        # Add the test LabMetaDataExtensionExample to the NWBFile
        nwbfile.add_lab_meta_data(lab_meta_data=metaObj)

        # Add video file
        video_ext_file = ImageSeries(
            name='behaviorVideo',
            description='Raw original video.',
            unit='n.a.',
            external_file=[os.path.join(meta.VideoPath[i],meta.VideoFile[i])],
            format='external',
            starting_time=0.0,
            rate=25.0,
        )

        nwbfile.add_acquisition(video_ext_file)

        # --- Add annotated behavior data

        # Make sure these files are downloaded with the same path configuration as listed in the metadata
        # Or adjust here as I did with telling the code to go up a level
        scoretab = pd.read_csv(os.path.join('..',meta.ScorePath[i],meta.ScoreFile[i])) # load up csv of annotations

        # change problematic column names
        scoretab.rename(columns={'Start (s)':'start'}, inplace=True)
        scoretab.rename(columns={'Stop (s)':'end'}, inplace=True)
        scoretab.rename(columns={'Duration (s)':'duration'}, inplace=True)
        scoretab.rename(columns={'Behavior type':'behavior_type'}, inplace=True)

        # --- Clean up small gaps in the behavior table
        # this assumes the score files have already been reviewed for major problems; everything at this stage should pass
        
        cleantab = set_event_ends(scoretab)
        
        # make NWB object corresponding to the annotation table
        behavior_intervals = TimeIntervals(name="annotated_behavior",
            description="Intervals of scored behavior.")

        behavior_intervals.add_column(name="behavior", description="The annotation from the ethogram.")
        behavior_intervals.add_column(name="duration", description="Duration of the behavior.")
        behavior_intervals.add_column(name="atype", description="Point or state event.")

        # populate table
        for i, start in enumerate(cleantab.start):
            end = cleantab.end[i]
            behav = cleantab.Behavior[i]
            atype = cleantab.behavior_type[i]
            dur = cleantab.duration[i]
            behavior_intervals.add_row(start_time=start,stop_time=end,behavior=behav,atype=atype,duration=dur)

        # add to NWB file
        nwbfile.add_time_intervals(behavior_intervals)

Nov22_Pair1_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001


Nov22_Pair15_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001


Nov22_Pair16_PPT.nwb


...

M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001


Nov22_Pair7_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001


Nov22_Pair8_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001


In [15]:
meta.FocalColor[i][0]

[245, 201, 39]

In [17]:
nwbfile

,start_time,stop_time,behavior,duration,atype
id,,,,,
0,21.433,21.433,Start assay,NaN,POINT
1,21.434,22.327,Center,0.893,STATE
2,22.328,28.215,Right,5.887,STATE
3,28.216,29.750,Interact Right,1.534,STATE
